<a href="https://colab.research.google.com/github/santoshrsarangi/tensorflow/blob/main/NLP/sarcasm_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import tensorflow as tf
from tensorflow import keras
import numpy as np
import urllib



In [ ]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
urllib.request.urlretrieve(url, 'sarcasm.json')

('sarcasm.json', <http.client.HTTPMessage at 0x7f41733d6ec0>)

In [ ]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

sentences = []
labels = []

In [ ]:
import pandas as pd

df = pd.read_json('sarcasm.json')


In [ ]:
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [ ]:
sentences=list(df['headline'])
labels=list(df['is_sarcastic'])

In [ ]:
len(sentences)

26709

In [ ]:
training_size=int(len(sentences) * 0.7)

In [ ]:
training_size

18696

In [ ]:
testing_size = int(len(sentences) * 0.15)

In [ ]:
training_sentences= sentences[:training_size]
training_labels = labels[:training_size]

val_sentences= sentences[training_size : len(sentences) - testing_size]
val_labels = labels[training_size : len(sentences) - testing_size]

test_sentences = sentences[-testing_size:]
test_labels = labels[-testing_size:]

In [ ]:
from keras.layers import TextVectorization

In [ ]:
text_vectorization = TextVectorization(output_mode="int", max_tokens=vocab_size, output_sequence_length=max_length)

In [ ]:
text_vectorization.adapt(training_sentences)

In [ ]:
vec_train_sentences = text_vectorization(training_sentences)

In [ ]:
vec_val_sentences = text_vectorization(val_sentences)

In [ ]:
vec_test_sentences = text_vectorization(test_sentences)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(keras.layers.LSTM(32)),
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           16000     
                                                                 
 bidirectional (Bidirectiona  (None, 64)               12544     
 l)                                                              
                                                                 
 dense (Dense)               (None, 120)               7800      
                                                                 
 dense_1 (Dense)             (None, 1)                 121       
                                                                 
Total params: 36,465
Trainable params: 36,465
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("1.h5", save_best_only=True)
]

In [ ]:
history = model.fit(vec_train_sentences, np.array(training_labels),
                    validation_data = (vec_val_sentences, np.array(val_labels)),
                    callbacks=callbacks,
                    epochs=32)

Epoch 1/32
585/585 [==============================] - 10s 17ms/step - loss: 0.3622 - accuracy: 0.8370 - val_loss: 0.3749 - val_accuracy: 0.8298
Epoch 2/32
585/585 [==============================] - 9s 16ms/step - loss: 0.3350 - accuracy: 0.8490 - val_loss: 0.3774 - val_accuracy: 0.8303
Epoch 3/32
585/585 [==============================] - 9s 15ms/step - loss: 0.3141 - accuracy: 0.8611 - val_loss: 0.3699 - val_accuracy: 0.8320
Epoch 4/32
585/585 [==============================] - 9s 15ms/step - loss: 0.3001 - accuracy: 0.8697 - val_loss: 0.3820 - val_accuracy: 0.8300
Epoch 5/32
585/585 [==============================] - 8s 14ms/step - loss: 0.2897 - accuracy: 0.8722 - val_loss: 0.3836 - val_accuracy: 0.8310
Epoch 6/32
585/585 [==============================] - 9s 15ms/step - loss: 0.2809 - accuracy: 0.8784 - val_loss: 0.4023 - val_accuracy: 0.8233
Epoch 7/32
585/585 [==============================] - 8s 14ms/step - loss: 0.2721 - accuracy: 0.8820 - val_loss: 0.4109 - val_accuracy: 0.814

In [ ]:
test_sentences[2]

'10 secrets of irresistible people'

In [ ]:
print(text_vectorization(test_sentences[2]))

tf.Tensor(
[152   1   3   1  64   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0], shape=(120,), dtype=int64)


In [ ]:
print(model.predict(text_vectorization(["this is example of laziness", "My neighbors listen to good music whether they like it or not"])))

1/1 [==============================] - 0s 124ms/step
[[0.00130517]
 [0.99801743]]
